<a href="https://colab.research.google.com/github/econdatatech/AIML429/blob/main/PGM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install -q pgmpy
from itertools import combinations
import pandas
import networkx as nx
from sklearn.metrics import f1_score
import numpy as np
from pgmpy.estimators import HillClimbSearch, ExhaustiveSearch,K2Score, BicScore, BDeuScore,MmhcEstimator,BDsScore, PC
from pgmpy.estimators.CITests import chi_square

import csv

#Pet and Prey

In [42]:
samplesp=pandas.DataFrame(pandas.read_csv("https://raw.githubusercontent.com/econdatatech/AIML429/main/pet-prey-temp.txt", sep='\t', lineterminator='\r',converters={'pet':lambda x:x.replace('\n','')})[:-1])

In [43]:
samplesp

,pet,prey,temp
0,dog,skink,cold
1,dog,mouse,warm
2,dog,mouse,warm
3,dog,skink,warm
4,dog,skink,cold
...,...,...,...
995,dog,sparrow,cold
996,dog,mouse,warm
997,dog,skink,cold
998,dog,skink,cold


In [44]:
# What 3-node belief net structure(s) would best match the dependencies exhibited by this data? (Justify your answer,
#i.e. are there independencies? conditional or otherwise? Does observation of one induce
#dependencies among the others?)

es = ExhaustiveSearch(samplesp,scoring_method=K2Score(samplesp))
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())


es = ExhaustiveSearch(samplesp,scoring_method=BicScore(samplesp))
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())

[('pet', 'prey'), ('pet', 'temp')]

All DAGs by score:
-2475.3085773118005 [('pet', 'prey'), ('pet', 'temp')]
-2475.3379855966637 [('pet', 'prey'), ('temp', 'pet')]
-2475.4812679125957 [('pet', 'prey')]
-2475.5316868232476 [('pet', 'temp'), ('prey', 'pet')]
-2475.704377424043 [('prey', 'pet')]
-2477.7280244022786 [('prey', 'pet'), ('temp', 'pet')]
-2477.930560367481 [('pet', 'temp')]
-2477.9599686523443 [('temp', 'pet')]
-2478.1032509682764 []
-2479.7608693619272 [('pet', 'prey'), ('prey', 'temp')]
-2479.819566924289 [('prey', 'pet'), ('temp', 'prey')]
-2479.9839788733743 [('prey', 'temp'), ('prey', 'pet')]
-2481.355126466882 [('pet', 'prey'), ('pet', 'temp'), ('temp', 'prey')]
-2481.3845347517454 [('pet', 'prey'), ('temp', 'pet'), ('temp', 'prey')]
-2481.5278170676775 [('pet', 'prey'), ('temp', 'prey')]
-2481.6582921762706 [('pet', 'prey'), ('pet', 'temp'), ('prey', 'temp')]
-2481.843213902525 [('prey', 'pet'), ('temp', 'pet'), ('temp', 'prey')]
-2481.8814016877177 [('pet', 'temp'), (

In [49]:
est = PC(samplesp)
skel, seperating_sets = est.build_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, seperating_sets)
print("PDAG edges:       ", pdag.edges())

model = pdag.to_dag()
print("DAG edges:        ", model.edges())

print(seperating_sets)

print(est.estimate(significance_level=0.01).edges())


##Validate the above results individually with test for INDEPENDENCE!
print(chi_square(X='pet', Y='prey', Z=[], data=samplesp, significance_level=0.01))
print(chi_square(X='pet', Y='prey', Z=['temp'], data=samplesp, significance_level=0.01))
print(chi_square(X='prey', Y='pet', Z=[], data=samplesp, significance_level=0.01))
print(chi_square(X='prey', Y='pet', Z=['temp'], data=samplesp, significance_level=0.01))

print(chi_square(X='temp', Y='pet', Z=[], data=samplesp, significance_level=0.01))
print(chi_square(X='temp', Y='pet', Z=['prey'], data=samplesp, significance_level=0.01))
print(chi_square(X='pet', Y='temp', Z=[], data=samplesp, significance_level=0.01))
print(chi_square(X='pet', Y='temp', Z=['prey'], data=samplesp, significance_level=0.01))
print(chi_square(X='temp', Y='prey', Z=[], data=samplesp, significance_level=0.01))
print(chi_square(X='temp', Y='prey', Z=['pet'], data=samplesp, significance_level=0.01))
print(chi_square(X='prey', Y='temp', Z=[], data=samplesp, significance_level=0.01))
print(chi_square(X='prey', Y='temp', Z=['pet'], data=samplesp, significance_level=0.01))

  0%|          | 0/5 [00:00<?, ?it/s]

Undirected edges:  [('pet', 'prey')]
PDAG edges:        [('prey', 'pet'), ('pet', 'prey')]
DAG edges:         [('pet', 'prey')]
{frozenset({'temp', 'pet'}): (), frozenset({'temp', 'prey'}): ()}


  0%|          | 0/5 [00:00<?, ?it/s]

[('pet', 'prey')]
False
False
False
False
True
True
True
True
True
True
True
True


/usr/local/lib/python3.10/dist-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/usr/local/lib/python3.10/dist-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/usr/local/lib/python3.10/dist-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/usr/local

#Wine and Shirt

In [46]:
samplesw=pandas.DataFrame(pandas.read_csv("https://raw.githubusercontent.com/econdatatech/AIML429/main/Meal-Shirt-Wine.txt", sep=' ', lineterminator='\r',converters={'Wine':lambda x:x.replace('\n','')})[:-1])
samplesw

,Wine,Meal,ShirtSleeves
0,white,lamb,long
1,red,lamb,short
2,red,vege,long
3,white,vege,short
4,white,vege,long
...,...,...,...
995,white,chicken,short
996,red,lamb,short
997,white,vege,long
998,red,chicken,long


In [47]:
es = ExhaustiveSearch(samplesw,scoring_method=K2Score(samplesw))
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())


es = ExhaustiveSearch(samplesw,scoring_method=BicScore(samplesw))
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())

[('Meal', 'ShirtSleeves'), ('Wine', 'ShirtSleeves')]

All DAGs by score:
-2711.295824520689 [('Meal', 'ShirtSleeves'), ('Wine', 'ShirtSleeves')]
-2713.721729316334 [('Meal', 'Wine'), ('ShirtSleeves', 'Wine')]
-2714.0235409813204 [('ShirtSleeves', 'Meal'), ('Wine', 'Meal'), ('Wine', 'ShirtSleeves')]
-2714.0642322347358 [('ShirtSleeves', 'Wine'), ('ShirtSleeves', 'Meal'), ('Wine', 'Meal')]
-2714.6978409030094 [('Meal', 'ShirtSleeves'), ('Wine', 'Meal'), ('Wine', 'ShirtSleeves')]
-2714.9769864853815 [('Meal', 'Wine'), ('ShirtSleeves', 'Wine'), ('ShirtSleeves', 'Meal')]
-2715.0871144893677 [('Meal', 'ShirtSleeves'), ('Meal', 'Wine'), ('Wine', 'ShirtSleeves')]
-2715.244682796858 [('Meal', 'ShirtSleeves'), ('Meal', 'Wine'), ('ShirtSleeves', 'Wine')]
-2715.4848500861426 [('Wine', 'ShirtSleeves')]
-2715.525541339558 [('ShirtSleeves', 'Wine')]
-2716.74010725519 [('ShirtSleeves', 'Meal'), ('Wine', 'ShirtSleeves')]
-2716.7807985086056 [('ShirtSleeves', 'Wine'), ('ShirtSleeves', 'Meal')]
-2717.048

In [50]:
est = PC(samplesw)
skel, seperating_sets = est.build_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, seperating_sets)
print("PDAG edges:       ", pdag.edges())

model = pdag.to_dag()
print("DAG edges:        ", model.edges())

print(seperating_sets)

##Validate the above results individually with test for INDEPENDENCE!
print(chi_square(X='Wine', Y='Meal', Z=[], data=samplesw, significance_level=0.01)) #T
print(chi_square(X='Wine', Y='Meal', Z=['ShirtSleeves'], data=samplesw, significance_level=0.01))
print(chi_square(X='Meal', Y='Wine', Z=[], data=samplesw, significance_level=0.01)) #T
print(chi_square(X='Meal', Y='Wine', Z=['ShirtSleeves'], data=samplesw, significance_level=0.01))

print(chi_square(X='ShirtSleeves', Y='Wine', Z=[], data=samplesw, significance_level=0.01))
print(chi_square(X='ShirtSleeves', Y='Wine', Z=['Meal'], data=samplesw, significance_level=0.01))
print(chi_square(X='Wine', Y='ShirtSleeves', Z=[], data=samplesw, significance_level=0.01))
print(chi_square(X='Wine', Y='ShirtSleeves', Z=['Meal'], data=samplesw, significance_level=0.01))
print(chi_square(X='ShirtSleeves', Y='Meal', Z=[], data=samplesw, significance_level=0.01))  #T
print(chi_square(X='ShirtSleeves', Y='Meal', Z=['Wine'], data=samplesw, significance_level=0.01))
print(chi_square(X='Meal', Y='ShirtSleeves', Z=[], data=samplesw, significance_level=0.01))  #T
print(chi_square(X='Meal', Y='ShirtSleeves', Z=['Wine'], data=samplesw, significance_level=0.01))

  0%|          | 0/5 [00:00<?, ?it/s]

Undirected edges:  [('Wine', 'ShirtSleeves')]
PDAG edges:        [('Wine', 'ShirtSleeves'), ('ShirtSleeves', 'Wine')]
DAG edges:         [('ShirtSleeves', 'Wine')]
{frozenset({'Wine', 'Meal'}): (), frozenset({'ShirtSleeves', 'Meal'}): ()}
True
False
True
False
False
False
False
False
True
False
True
False


/usr/local/lib/python3.10/dist-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/usr/local/lib/python3.10/dist-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/usr/local/lib/python3.10/dist-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/usr/local